Pytorch Datasets and Dataloaders <br>
    https://pytorch.org/tutorials/beginner/basics/data_tutorial.html <br>
    https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html

Installing Pytorch and preparing the environment: <br>
From tool on Pytorch's installation website (https://pytorch.org/get-started/locally/) <br>
Use the command: conda install pytorch torchvision torchaudio pytorch-cuda=11.6 -c pytorch -c nvidia <br>
Will also need to install pandas, matplotlib, scikit-learn (sklearn), and seaborn with conda

The subcellular localization data is the neighborhood annotations for MCF7 cells from Orre et al.

In [ ]:
# Import packages
import re
from pathlib import Path
import numpy as np
import pandas as pd
import os
from pdb import set_trace
import matplotlib.pyplot as plt
import PIL #for images
import copy
import random

import seaborn as sns
import matplotlib.pyplot as plt

import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
from torchvision.io import read_image
from torch.utils.data import DataLoader
from torchvision import models
from torch import nn
from torchvision.transforms import Resize
import torchvision.transforms as T

In [ ]:
#Specifications

# Data splitting and path
ImagePathString = '/home/ubuntu/NeuralNetworks/Gene_Images/Krug_18by18Grid'
path2 = Path(ImagePathString)
ValFrac = 0.3

# Training and Validation Data Sets
ImageTransformationName = 'None'
ShuffleLabels = False
#   if you want to shuffle the labels to see if a model is actually training

if ImageTransformationName == 'GaussianBlur':
    blur_sigma = (1.1,1.1)
    # (0.9,0.9) is a decent value
    
# Model
ModelWeightSpefication = 'random'
if ModelWeightSpefication == 'random':
    ModelWeights = None
if ModelWeightSpefication == 'pretrained':
    ModelWeights = models.ResNet50_Weights.DEFAULT
    # models.ResNet50_Weights.DEFAULT is some pretrained set of values

# Optimizer
Weight_Decay = 0.0
#  penalizes for too many weights - helps prevent overfitting
Momentum = 0.9
#  a way to smooth noise that is passed to the optimizer, 0.9
#  momentum is deterimental without label smoothing
learner_rate = 5e-4
# 1e-3 seems to work well

# Loss Function
Label_Smoothing = 0.1
#  was 0.1
#  sets the target of the loss function to something greater than 0 and less than 1
#  helps prevent overfitting
AddLayerForWeights = True
# Makes the loss weights equal to the fraction of each category label
# Note, a layer is added to the model so the outputs of the model are equal to the number of categories
#     This seems like a bug becuase that should already be the case

# Run the model
epochs = 1000

In [ ]:
# Check to see if the GPU is available and store it as a variable so tensors can be moved to it
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print('Is cuda available?:', torch.cuda.is_available())
print('cuda version:', torch.version.cuda)
dev = "cuda:0"

In [ ]:
# Define necessary classes and functions

# Create the CustomImageDataset class from the parent Dataset class
class CustomImageDataset(Dataset):
    def __init__(self, annotations_DF, img_dir, transform=None, target_transform=None,ImageExtension='.png'):
        self.img_labels = annotations_DF
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform
        self.ImageExtension = ImageExtension

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0] + self.ImageExtension)
        image = read_image(img_path)
        #image = PIL.Image.open(img_path)
        #    attempt to get around byte vs. float problem: https://pytorch.org/hub/pytorch_vision_resnet/
        #image = image.double()
        #image = plt.imread(img_path)
        #  used in troubleshooting torchvision vs matplatlib formats
        label = self.img_labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

# Create a function to output training and validation annotation data frames from a single annotations file
def GetTrainValAnnotDataFrames(annotations_file,img_dir,val_frac,ShuffleLabels=False):
    # Inputs:
    # annotations_file: a directory to an comma separated file with image filenames and their labels
    # img_dir: a directory to the folder containing the image files
    # val_frac: the fraction of samples used for validation
    
    # read the csv file with the sample annotations
    annotationsDF = pd.read_csv(annotations_file,header=None,sep='\t',dtype=str)
    
    # read the directory containing the images to get a list of filenames
    #     strip the file extension so the filenames match the annotations in annotationsDF
    FileList = os.listdir(img_dir)
    Files_Without_Extension = [os.path.splitext(file)[0] for file in FileList]
    
    # remove entries in the annotationsDF that are not in the list of images
    #     this is done so that the training and validation sets are divided correctly
    OverlappingIndices = [item in Files_Without_Extension for item in annotationsDF.iloc[:,0]]
    annotationsDF = annotationsDF.loc[OverlappingIndices,:]

    # Shuffle the labels if indicated to do so
    if ShuffleLabels:
        LabelList = annotationsDF.iloc[:,1].tolist()
        RandomLabels = copy.copy(LabelList)
        random.shuffle(RandomLabels)
        annotationsDF.iloc[:,1] = RandomLabels
     
    # remove unclassified indices for Zhuoheng's images (should not have to, but loss function does not have 5 when left in)
    # NotUnclassifiedRowIndices = annotationsDF.iloc[:,1]!='Unclassified'
    # annotationsDF = annotationsDF.loc[NotUnclassifiedRowIndices,:]
    
    
    
    unique_labels = np.unique(annotationsDF.iloc[:,1])
    ValidationDF = pd.DataFrame()
    TrainingDF = pd.DataFrame()
    for label in unique_labels:
        Subset_DF = annotationsDF.loc[annotationsDF.iloc[:,1]==label,:]
        n_validation = int(np.floor(val_frac * len(Subset_DF.iloc[:,0])))
        Validation_Subset = Subset_DF.sample(n=n_validation,ignore_index=True,random_state=42)
        ValidationDF = pd.concat(objs=[ValidationDF,Validation_Subset],axis=0)
        
    Training_rows = [gene not in ValidationDF.iloc[:,0].to_numpy() for gene in annotationsDF.iloc[:,0].to_numpy()]
    TrainingDF = annotationsDF.loc[Training_rows,:]
    
    return(ValidationDF, TrainingDF, unique_labels)

# Function to convert a torchvision tensor to the default format for matplotlib
    # matplotlib.pyplot.imshow interprets an RGB vector as shape (N,M,3) 
    # torchvision.io.read_image reads an RGB image as shape (3,N,M)
def ShowTorchvisionImage(ImageTensor):
    ImageTensor_t1 = ImageTensor.transpose(1,2)
    ImageTensor_t2 = ImageTensor_t1.transpose(0,2)
    plt.imshow(ImageTensor_t2)
    
def LabelArray_to_Tensor(input_array,labels_unique):
    # converts a label array of strings to a tensor of floats
    # Inputs:
    #   input_array: the array of strings that are labels
    #   labels_unique: an ordered list of the unique labels possible for the data set
    # Outputs:
    #   output_tensor: a tensor of floats, one unique float corresponding to each label
    
    int_label = np.arange(0,len(labels_unique))
    i = 0
    for label in labels_unique:
        indices = [label == item for item in input_array]
        input_array[indices] = int_label[i]
        i = i+1

    input_array = input_array.astype(float)
    output_tensor = torch.as_tensor(input_array)
    output_tensor = output_tensor.long()
  
    return(output_tensor)

# Function for training the weights of the model
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss

        
        # convert to float32 to avoid error stating byte expected but found float
        X = X.to(torch.float32)
        X = X.to(dev)
        pred = model(X)
        
        # convert the labels into a tensor of intigers and compute the loss
        y = np.asarray(y)
        y = LabelArray_to_Tensor(y,labels_unique)
        y = y.to(dev)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
             

    loss = loss.item()
    return(loss)
            
# Function for validating the weights of the model
def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            
            # convert to float32 to avoid error stating byte expected but found float
            X = X.to(torch.float32)
            X = X.to(dev)
            pred = model(X)
            
            # convert the labels into a tensor of intigers and compute the loss
            y = np.asarray(y)
            y = LabelArray_to_Tensor(y,labels_unique)
            y = y.to(dev)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    
    return(test_loss,correct)

In [ ]:
# Split the samples up into training and validation
Val_DF, Train_DF, unique_labels = GetTrainValAnnotDataFrames(annotations_file='../Label_Files/MCF7_SubCell_Barcode.txt',img_dir=path2,val_frac=ValFrac,ShuffleLabels=ShuffleLabels)
labels_unique = np.unique(Train_DF.iloc[:,1])

TrainingData = CustomImageDataset(annotations_DF=Train_DF,img_dir=path2)
ValData = CustomImageDataset(annotations_DF=Val_DF,img_dir=path2)


# Create the data sets with transformations from the training and validation set data frames defined above
if ImageTransformationName == 'None':
    TrainingData = CustomImageDataset(annotations_DF=Train_DF,img_dir=path2)
    ValData = CustomImageDataset(annotations_DF=Val_DF,img_dir=path2)

if ImageTransformationName == 'SpecifiedReorder':
    TrainingData = CustomImageDataset(annotations_DF=Train_DF,img_dir=path2,transform=SpecifiedReorder(sample_reorder_array))
    ValData = CustomImageDataset(annotations_DF=Val_DF,img_dir=path2,transform=SpecifiedReorder(sample_reorder_array))
    
if ImageTransformationName == 'GaussianBlur':
    transform = T.GaussianBlur(kernel_size=(7, 13), sigma=blur_sigma)
    TrainingData = CustomImageDataset(annotations_DF=Train_DF,img_dir=path2,transform=transform)
    ValData = CustomImageDataset(annotations_DF=Val_DF,img_dir=path2,transform=transform)
    
if ImageTransformationName == 'RandomCrop':
    transform = T.RandomCrop(size=(50, 50))
    TrainingData = CustomImageDataset(annotations_DF=Train_DF,img_dir=path2,transform=transform)
    ValData = CustomImageDataset(annotations_DF=Val_DF,img_dir=path2,transform=transform)

    
if ImageTransformationName == 'RandomRotation':
    transform = T.RandomRotation(degrees=(-10, 10))
    TrainingData = CustomImageDataset(annotations_DF=Train_DF,img_dir=path2,transform=transform)
    ValData = CustomImageDataset(annotations_DF=Val_DF,img_dir=path2,transform=transform)
    
if ImageTransformationName == 'Resize':
    transform = T.Resize(size=(50, 50))
    TrainingData = CustomImageDataset(annotations_DF=Train_DF,img_dir=path2,transform=transform)
    ValData = CustomImageDataset(annotations_DF=Val_DF,img_dir=path2,transform=transform)
    
# Other possible transformations
#TrainingData = CustomImageDataset(annotations_DF=Train_DF,img_dir=path2,transform=Resize([224,224]))
#TrainingData = CustomImageDataset(annotations_DF=Train_DF,img_dir=path2,transform=RandomPairReposition())

# Define Data Loaders
train_dataloader = DataLoader(TrainingData, batch_size=64, shuffle=True)
val_dataloader = DataLoader(ValData, batch_size=64, shuffle=True)


# Check to see if the number of training data points plus validation data points sum to total data points
print('sum of Val and Train Sets: ' + str(len(Val_DF.index)+len(Train_DF.index)))
n_files = len(os.listdir(path2))
print('num of samples: ' + str(n_files))

# Display a training image and its corresponding label.
train_features, train_labels = next(iter(train_dataloader))
#print(f"Feature batch shape: {train_features.size()}")
#print(f"Labels batch shape: {train_labels.size()}")
img = train_features[0].squeeze()
label = train_labels[0]
ShowTorchvisionImage(img)
print(f"Label: {label}")

In [ ]:
# Define the model

# initial model weights
model = models.resnet50(weights=ModelWeights)
#model = models.resnet50() #without initial weights defined - good for patients not re-arranged

# Add a final layer to the model where the output is equal to the number of classes (labels)
#     This is required to use loss weights
if AddLayerForWeights:
    model.fc = nn.Sequential(
        model.fc,
        nn.Linear(1000, len(unique_labels))
    )

model = model.to(dev)

# see the model architecture if desired
#model.parameters

In [ ]:
# Define the loss function
# This needs to be made general for the number of classes

loss_fn = nn.CrossEntropyLoss()
if AddLayerForWeights:
    # Weights for each class for loss function
    n_cytosol = float(sum(TrainingData.img_labels.iloc[:,1]=='Cytosol'))
    n_mitochondria = float(sum(TrainingData.img_labels.iloc[:,1]=='Mitochondria'))
    n_nuclear = float(sum(TrainingData.img_labels.iloc[:,1]=='Nuclear'))
    n_secretory = float(sum(TrainingData.img_labels.iloc[:,1]=='Secretory'))
    #n_unclassified = float(sum(TrainingData.img_labels.iloc[:,1]=='Unclassified'))
    n_total = len(TrainingData.img_labels.iloc[:,1])
    LossWeights = (torch.tensor([n_cytosol,n_mitochondria,n_nuclear,n_secretory])/n_total)**-1
    LossWeights = LossWeights.to(device)
    loss_fn = nn.CrossEntropyLoss(label_smoothing=Label_Smoothing,weight=LossWeights)

In [ ]:
# Define the optimizer

optimizer = torch.optim.SGD(model.parameters(), lr=learner_rate, weight_decay=Weight_Decay, momentum=Momentum)
#optimizer = torch.optim.SGD(model.parameters(), lr=learner_rate)

In [ ]:
# Run the model

PerformanceDF = pd.DataFrame(index=range(epochs),columns=['Epoch','TrainLoss','ValLoss','TrainAcc','ValAcc'])
print(f"{'Epoch':>5s}{'Train Loss':>13s}{'Val Loss':>11s}{'Train Accuracy':>17s}{'Val Accuracy':>15s}\n")


# The epoch of index 0 is before the first epoch - the model has not trained at all
#   note: the result after the last epoch is not recorded or printed
for t in range(epochs): 
    #train_loss = train_loop(train_dataloader, model, loss_fn, optimizer)
    val_loss,val_accuracy = test_loop(val_dataloader, model, loss_fn)
    train_loss2,train_accuracy = test_loop(train_dataloader, model, loss_fn)
    Epoch = t
    print(f"{str(Epoch):>5s}{train_loss2:>13f}{val_loss:>11f}{train_accuracy*100:>16f}%{val_accuracy*100:>14f}%")
    PerformanceDF.loc[t,:]=[Epoch,train_loss2,val_loss,train_accuracy*100,val_accuracy*100]
    train_loss = train_loop(train_dataloader, model, loss_fn, optimizer)
    
    # note: I am not sure how to load the MSD into the model
    if Epoch == 0:
        highest_val_accuracy = 0 
        MSD0 = copy.deepcopy(model.state_dict())
    if val_accuracy > highest_val_accuracy:
        highest_val_accuracy = val_accuracy
        MSD_Best = copy.deepcopy(model.state_dict())
    
print("Done!")

In [ ]:
# Plot the training and validation accuracies as a function of the epoch number

# Convert to a long data frame
LongDF = pd.melt(frame=PerformanceDF,value_vars=['ValAcc','TrainAcc'],id_vars='Epoch')

# Make and format the plot
plot = sns.lineplot(data=LongDF,x='Epoch',y='value',hue='variable')
sns.despine(bottom = False, left = False)
plot.set(ylim=(0, None))
plot.set(xlim=(0, 70))
vals = plot.get_yticks()
plot.set_yticklabels(plot.get_yticks(), size = 15)
plot.set_xticklabels(plot.get_xticks(), size = 15)
plot.set_yticklabels(['{:.0f}%'.format(x) for x in vals])
#plot.set_xticklabels(['{}'.format(int(x)) for x in domains])

if not ShuffleLabels:
    plt.savefig("../outputs/TrueLabels_LineTrace.pdf", format='pdf')

if ShuffleLabels:
    plt.savefig("../outputs/ShuffledLabels_LineTrace.pdf", format='pdf')



In [ ]:
print('Best recorded epoch validation accuracy: ' + str(round(100*highest_val_accuracy,2)))

BestModel = copy.deepcopy(model)
BestModel.load_state_dict(MSD_Best)
val_loss,best_val_accuracy = test_loop(val_dataloader, BestModel, loss_fn)
print('Best epoch recalculated validation accuracy: ' + str(round(100*best_val_accuracy,2)))

FirstModel = copy.deepcopy(model)
FirstModel.load_state_dict(MSD0)
val_loss,first_val_accuracy = test_loop(val_dataloader, FirstModel, loss_fn)
print('First epoch validation Accuracy: ' + str(round(100*first_val_accuracy,2)))

In [ ]:
# build confusion matrix
#   https://christianbernecker.medium.com/how-to-create-a-confusion-matrix-in-pytorch-38d06a7f04b7

from sklearn.metrics import confusion_matrix
import seaborn as sn

y_pred = []
y_true = []

# iterate over test data
for X, y in val_dataloader:
    
        X = X.to(torch.float32)
        X = X.to(dev)
        output = model(X) # Feed Network

        output = (torch.max(torch.exp(output), 1)[1]).data.cpu().numpy()
        output_string = [unique_labels[i] for i in output]
        
        y_pred.extend(output_string) # Save Prediction
        
        labels = y
        y_true.extend(labels) # Save Truth

# constant for classes
classes = unique_labels

# Build confusion matrix
cf_matrix = confusion_matrix(y_true, y_pred)
cf_matrix_norm = np.zeros((len(classes),len(classes)))
for i in np.arange(0,len(classes)):
    cf_matrix_norm[i,:] = cf_matrix[i,:]/sum(cf_matrix[i,:])

df_cm = pd.DataFrame(cf_matrix_norm, index = [i for i in classes],
                     columns = [i for i in classes])
plt.figure(figsize = (6,5))
hm = sn.heatmap(df_cm, annot=True)
hm.set(xlabel='\n\nPrediction', ylabel='Label\n\n')


plt.savefig('output.png')

In [ ]:
# Validate Orientation of Confusion Matrix
#     Data labels go down the rows and predictions label the columns

# Print Confusion Matrix Row Sums
ConfusionMatrix_RowSums = [sum(cf_matrix[i,:]) for i in np.arange(0,len(unique_labels))]
print(ConfusionMatrix_RowSums)

# Print Quantity of Items with each label
ValDF_UniqueLabelSums = [sum(Val_DF.iloc[:,1]==classes[i]) for i in np.arange(0,len(unique_labels))]
print(ValDF_UniqueLabelSums)

Old Stuff Below!

In [ ]:
# Create a class for transforming the input image by a specified reordering of adjacent pairs


# The custom class must have the parent class of object because that is what is expected by the CustomImageDataset object
class SpecifiedReorder(object):
    
    def __init__(self,InputOrder):
#         self.img = InputTensor
         self.ReOrder = InputOrder
#         self.NewImage = self.transform()
    
    # The InputTensor is what is passed to the call of the class - this is probably behaviour of the object parent class
    def __call__(self,InputTensor):
        # Krista
        # Inputs:
        #   InputTensor: An n x n x 3 tensor
        #   InputOrder: An array of length (n x n)/2

        # Outputs:
        #   A n x n x 3 tensor where each n x n layer has been reordered in the same way
        #     consecutive pairs have been placed in the specified locations, still next to each other and both on the same row
        
        
        # Determine the number of measurements in the tensor
        TensorDimensions = np.shape(InputTensor)
        array_n = TensorDimensions[1] * TensorDimensions[2]
        matrices_n = TensorDimensions[0]

        # Specify the desired order of samples
        sample_order = self.ReOrder
        sample_order_indices = 2*sample_order

        
        TransformedArray = np.zeros((InputTensor.size()[0],InputTensor.size()[1],InputTensor.size()[2]))
        TransformedArray = TransformedArray.astype(np.uint8)

        for matrix_i in np.arange(0,matrices_n):

            # Define the matrix to transform
            matrix = InputTensor[matrix_i]

            # Convert the matrix to an array
            array = np.array(matrix.flatten())

            # initialize the full diagonal of the transformation matrix
            reordered_indices = np.zeros(array_n)
            for i in np.arange(array_n/2):
                i = i.astype(int)
                # the first index in a pair will be from the randomly ordered even indices
                reordered_indices[2*i] = sample_order_indices[i]
                # the second index in a pair will be the odd number one above the even index number
                reordered_indices[2*i+1] = sample_order_indices[i] + 1

            # Convert indices to values in the original array
            transformed_array = np.array([array[int(i)] for i in reordered_indices])

            # Convert the array back to a torch tensor with its original shape and data type
            array_2D = np.reshape(transformed_array,np.shape(matrix))
            array_2D = array_2D.astype(np.uint8)
            TransformedArray[matrix_i,] = array_2D

        TransformedTensor = torch.from_numpy(TransformedArray)
        #self.NewImage = TransformedTensor

        return(TransformedTensor)

In [ ]:
# Function to transform image
#   Randomly repositions adjacent pairs of pixels
#   Done so model does not use patient order
def RandomPairRepositionFunc(InputTensor):
    # Krista
    # Inputs:
    #   A n x n x 3 tensor

    # Outputs:
    #   A n x n x 3 tensor where each n x n layer has been reordered in the same way
    #     consecutive pairs have been randomly placed in a different location, still next to each other and both on the same row

    # Determine the number of measurements in the tensor
    TensorDimensions = np.shape(InputTensor)
    array_n = TensorDimensions[1] * TensorDimensions[2]
    matrices_n = TensorDimensions[0]

    # initialize an array to be the diagonal of the transformation matrix
    diagonal_n = int(array_n/2)

    # fill in the diagonal array numerators as randomly ordered indices from 0 to 1/2 the number of indices in data array
    #     multiply randomly selected indices by 2 because random indices are the first in random adjacent paris selected from the data array
    diagonal = 2*np.random.choice(np.arange(array_n/2), size=diagonal_n, replace=False, p=None)


    TransformedArray = np.zeros((3,14,14))
    TransformedArray = TransformedArray.astype(np.uint8)

    for matrix_i in np.arange(0,matrices_n):

        # Define the matrix to transform
        matrix = InputTensor[matrix_i]

        # Convert the matrix to an array
        array = np.array(matrix.flatten())

        # initialize the full diagonal of the transformation matrix
        diagonal2 = np.zeros(array_n)
        for i in np.arange(array_n/2):
            i = i.astype(int)
            # the first index in a pair will be from the randomly ordered even indices
            diagonal2[2*i] = diagonal[i]
            # the second index in a pair will be the odd number one above the even index number
            diagonal2[2*i+1] = diagonal[i] + 1

        # Convert indices to values in the original array
        diagonal_values = np.array([array[int(i)] for i in diagonal2])

        # Perform elementwise division of the re-ordered values taken from the original array with the original values in their original order
        diagonal_values2 = np.divide(diagonal_values,array)

        # Build the transormation matrix by setting the above values as the diagonal with all other values 0 in a square matrix
        CoefficientMatrix = np.zeros((array_n,array_n))
        np.fill_diagonal(CoefficientMatrix,diagonal_values2)

        # Transform the original data array
        transformed_array = np.matmul(array,CoefficientMatrix)

        # Convert the array back to a torch tensor with its original shape and data type
        array_2D = np.reshape(transformed_array,np.shape(matrix))
        array_2D = array_2D.astype(np.uint8)
        TransformedArray[matrix_i,] = array_2D

    TransformedTensor = torch.from_numpy(TransformedArray)

    return(TransformedTensor)

In [ ]:
# Create a function to calculate the distance between two adjacent points in an image 
#   The image is specified by a 3D tensor
def DistCalc(Tensor,k,j):
    # Inputs:
    #   k specifies which pair within a column
    #   j specifies the row
    #   Tensor is the input image 3D tensor
    
    col = 2*k
    term1 = (Tensor[0][j][col].double() - Tensor[0][j][col+1].double())**2
    term2 = (Tensor[1][j][col].double() - Tensor[1][j][col+1].double())**2
    term3 = (Tensor[2][j][col].double() - Tensor[2][j][col+1].double())**2
    dist = (term1 + term2 + term3)**0.5
    dist_float = dist.item()
    return(dist_float)

# Split the samples up into training and validation
Val_DF, Train_DF, unique_labels = GetTrainValAnnotDataFrames(annotations_file='MCF7_SubCell_Barcode.txt',img_dir=path2,val_frac=ValFrac)
labels_unique = np.unique(Train_DF.iloc[:,1])

TrainingData = CustomImageDataset(annotations_DF=Train_DF,img_dir=path2)
ValData = CustomImageDataset(annotations_DF=Val_DF,img_dir=path2)

# Compute the distances between protein and mRNA expression values for every gene across all patients
#     each gene correpsonds to an image
#     each patient corresponds to a protein-mRNA pixel pair in the image

n_images = len(TrainingData)+len(ValData)
n_TrnData = len(TrainingData)
n_pixels_edge = np.shape(TrainingData[0][0])[1]
n_pixels = np.shape(TrainingData[0][0])[1]*np.shape(TrainingData[0][0])[2]
DistArray = np.zeros((int(n_pixels/2),n_images),dtype=float)
for i in np.arange(0,n_images):
    if i%1000==0:
        print(i)
    if i < n_TrnData:
        Tensor = TrainingData[i][0]
    if i >= n_TrnData:
        Tensor = ValData[i-n_TrnData][0]
    DistTens0 = [DistCalc(Tensor,k,j) for j in np.arange(0,n_pixels_edge) for k in np.arange(0,int(n_pixels_edge/2))]
    DistArray[:,i] = DistTens0

# Compute the average protein-mRNA distance across genes for each patient
AvgDistances = np.sum(DistArray,1)/n_images

# Rank the patients from lowest to highest distance
sample_reorder_array = np.argsort(AvgDistances)

In [ ]:
ShowTorchvisionImage(RandomPairRepositionFunc(img))

In [ ]:
file_list = os.listdir(ImagePathString)
len(file_list)
file_list.sort()
print(file_list)

In [ ]:
os.remove(ImagePathString+'/.DS_Store')

Got 40% accuracy with significant higher than null true positive rates for every category with weight decay = 0, momentum=0 and label weighting. 1e-3 learning rate until ~90% accurate on training data, then 1e-4 learning rate.

Got a lower accuracy, but improved true positive rates for the other categories (not cytosol) when weight decay and momentum were added. Adding blur improved true positive rates and true negative rates, and slightly improved overall accuracy much (something to do with most genes being cytsolic and nuclear?). Blur sigma of (0.9,0.9) worked the best so far, but may continue to improve if I raise it more.

Get hightest validation accuracy there are not transformations, when patients have not been re-ordered, there is no label smoothing or momentum, no layer is added to the model, and there are no weights on the loss function, and no default weights are passed to the model. (72-73%)

Adding loss function weights works well too, brings accuracy to 70%.

Notes on identifying Zhuoheng's pictures:
- patient ID's provided by color
-   44% accuracy after 10 epochs, with little improvement after 1st epoch (42% accuracy)
-   bias towards cytosol

In [ ]:
img.size()

In [ ]:
from scipy import misc

import numpy as np
import matplotlib.pyplot as plt

#Loads an arbitrary RGB image from the misc library
rgbImg = img.detach().clone()
img1 = copy.copy(img)

#Display out the original RGB image
#ShowTorchvisionImage(img)

# doc on shuffle: multi-dimensional arrays are only shuffled along the first axis
# so let's make the image an array of (N,3) instead of (m,n,3)

rndImg2 = np.reshape(rgbImg, (rgbImg.shape[1] * rgbImg.shape[2], rgbImg.shape[0]))
# this like could also be written using -1 in the shape tuple
# this will calculate one dimension automatically
# rndImg2 = np.reshape(rgbImg, (-1, rgbImg.shape[2]))



#now shuffle
np.random.shuffle(rndImg2)

#and reshape to original shape
rdmImg = np.reshape(rndImg2, rgbImg.shape)

ShowTorchvisionImage(img)

In [ ]:
sum(img[0] != 0)